<a href="https://colab.research.google.com/github/rajeevradnair/openai/blob/main/openai_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install openai==1.55.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [8]:
pip install httpx==0.23.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.7
    Uninstalling httpcore-1.0.7:
      Successfully uninstalled httpcore-1.0.7
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled httpx-0.27.2


In [2]:
#pip install httpx==0.23.0
#pip install openai==1.55.3

from openai import OpenAI
from google.colab import userdata

client=OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
client

In [4]:
trainingFile=client.files.create(
    file=open("train.jsonl","rb"),
    purpose="fine-tune"
)
trainingFile

FileObject(id='file-MiiD6VGFtYC5oMzKhyW6gp', bytes=6260, created_at=1733459402, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [5]:
validationFile=client.files.create(
    file=open("val.jsonl","rb"),
    purpose="fine-tune"
)
validationFile

FileObject(id='file-L61BaEeataqRLgGxGrFdUq', bytes=6185, created_at=1733459451, filename='val.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [7]:
trainingJob=client.fine_tuning.jobs.create(
    model="gpt-3.5-turbo",
    training_file=trainingFile.id,
    validation_file=validationFile.id,
    hyperparameters={"n_epochs": 3}
)
trainingJob

FineTuningJob(id='ftjob-9xfjDUMEFooIi89eQAMlprVz', created_at=1733459622, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-IlRHBLOqahdoneggd2qVlPhD', result_files=[], seed=1667389821, status='validating_files', trained_tokens=None, training_file='file-MiiD6VGFtYC5oMzKhyW6gp', validation_file='file-L61BaEeataqRLgGxGrFdUq', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [10]:
trainingJob.status

'validating_files'

In [18]:
trainingJob=client.fine_tuning.jobs.retrieve(
    fine_tuning_job_id=trainingJob.id
)
trainingJob.status

'succeeded'

In [16]:
trainingJobEvents=client.fine_tuning.jobs.list_events(
    fine_tuning_job_id=trainingJob.id
)
trainingJobEvents.data

list

In [19]:
trainingJob.fine_tuned_model

'ft:gpt-3.5-turbo-0125:personal::AbKQcel0'

In [23]:
model=client.models.retrieve(trainingJob.fine_tuned_model)
model

Model(id='ft:gpt-3.5-turbo-0125:personal::AbKQcel0', created=1733460026, object='model', owned_by='user-sobrllqvoto8bb3pkewedbi1')

In [27]:
response=client.chat.completions.create(
    model=trainingJob.fine_tuned_model,
    messages=[
        {"role": "system", "content": "You are a smart assistant that responds in Latin"},
        {"role": "user", "content": "Who is a friend?"}
    ]
)
response.choices[0].message.content

'Amicus est tamquam alter idem.'

In [33]:
messages=[]
messages.append({"role": "system", "content": "You are a smart assistant who responds in Latin"})

while True:
  s=input("Enter question:")
  if s.strip().lower()=="exit":
    break
  messages.append({"role": "user", "content": s})
  response=client.chat.completions.create(
      model=trainingJob.fine_tuned_model,
      messages=messages
  )
  messages.append({"role": "assistant", "content": response.choices[0].message.content})
  print(response.choices[0].message.content)



Enter question:     exIT
